<a href="https://colab.research.google.com/github/ji-in/PyTorch/blob/main/neural_networks_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


Neural Networks
===============


신경망은 `torch.nn` 패키지를 사용해서 만든다.  

`nn`은 모델들을 정의하고 미분하기 위해 `autograd`에 의존한다.  
`nn.Module`은 레이어와 `output`을 반환하는 `forward(input)`함수를 가진다.  

예를들어, 숫자 이미지를 분류하는 네트워크를 보자.  

<!--.. figure:: /_static/img/mnist.png
   :alt: convnet

   convnet-->

이것은 간단한 feed-forward 네트워크이다. 입력을 받아 여러 레이어를 차례로 통과시키고, 마지막으로 출력을 한다.

신경망을 위한 일반적인 훈련 절차는 다음을 따른다:  

- 학습 가능한 파라미터(또는 가중치)를 가지는 신경망을 정의한다.  
- 입력 데이터셋에 대해 반복한다.  
- 입력을 네트워크에 통과시킨다.  
- loss를 계산한다. (출력이 정답과 얼마나 차이가 나는지 계산한다.)  
- 네트워크의 파라미터에 기울기를 역전파한다.  
- 다음의 규칙을 사용해서 네트워크의 가중치를 갱신한다: ``weight = weight - learning_rate * gradient``  

Define the network
------------------

네트워크를 정의해보자.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


`forward` 함수를 정의해야하고, `autograd`를 사용할 수 있도록 `backward` 함수(여기서 기울기 계산)는 자동으로 정의된다.

모델의 학습 가능한 파라미터는 `net.parameters()`에 의해 반환된다.


In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


랜덤으로 만들어진 32x32 입력을 네트워크에 넣어보자.  
Note: 이 네트워크(LeNet)에 들어가는 입력의 크기는 32x32이다. MNIST 데이터셋에서 이 네트워크를 사용하기 위해, MNIST 데이터셋의 이미지 크기를 32x32로 맞춰야 한다.

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0579, -0.0501, -0.0871,  0.0122,  0.1219,  0.1780,  0.0423,  0.0277,
          0.0597, -0.0357]], grad_fn=<AddmmBackward>)


모든 파라미터의 기울기 버퍼를 0으로 만들고 랜덤으로 만든 기울기로 역전파를 수행하자:

In [ ]:
net.zero_grad()
out.backward(torch.randn(1, 10))

#### Note
`torch.nn`은 샘플의 mini-batches만 입력으로 사용한다. 단일 이미지는 사용하지 않는다.  예를 들어, `nn.Conv2d`는 4D Tensor인 `nSamples x nChannels x Height x Width`를 입력으로 받는다.  

만약 샘플 하나만 가지고 있다면, `input.unsqueeze(0)`을 써서 가짜 배치 차원을 추가해야 한다.  

계속 진행하기 전에, 지금까지 공부했던 것들을 요약해보자.  

**요약:**
  - `torch.Tensor` - `backward()`와 같은 autograd 연산을 도와주는 다차원 배열이다. 또한 텐서 형태로 기울기를 가지고 있는다.
  - `nn.Module` - 신경망 모듈이다. 파라미터를 GPU로 이동하고, 내보내고, 불러오기 위해 도움을 받는 파라미터들을 캡슐화하기 편리한 방법이다.
  -  ``nn.Parameter`` - A kind of Tensor, that is *automatically
     registered as a parameter when assigned as an attribute to a*
     ``Module``.
  - `nn.Parameter` - 
  -  ``autograd.Function`` - Implements *forward and backward definitions
     of an autograd operation*. Every ``Tensor`` operation creates at
     least a single ``Function`` node that connects to functions that
     created a ``Tensor`` and *encodes its history*.
  - `autograd.Function` - autograd 연산의 forward와 backward를 수행한다. 모든 `Tensor` 연산은 적어도 한 개의 `Function` 노드를 생성한다. `Tensor`를 만들었고 

**At this point, we covered:**
  -  Defining a neural network
  -  Processing inputs and calling backward

**Still Left:**
  -  Computing the loss
  -  Updating the weights of the network

Loss Function
-------------
A loss function takes the (output, target) pair of inputs, and computes a
value that estimates how far away the output is from the target.

There are several different
`loss functions <https://pytorch.org/docs/nn.html#loss-functions>`_ under the
nn package .
A simple loss is: ``nn.MSELoss`` which computes the mean-squared error
between the input and the target.

For example:



In [ ]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Now, if you follow ``loss`` in the backward direction, using its
``.grad_fn`` attribute, you will see a graph of computations that looks
like this:

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:



In [ ]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop
--------
To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients though, else gradients will be
accumulated to existing gradients.


Now we shall call ``loss.backward()``, and have a look at conv1's bias
gradients before and after the backward.



In [ ]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

Now, we have seen how to use loss functions.

**Read Later:**

  The neural network package contains various modules and loss functions
  that form the building blocks of deep neural networks. A full list with
  documentation is `here <https://pytorch.org/docs/nn>`_.

**The only thing left to learn is:**

  - Updating the weights of the network

Update the weights
------------------
The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):

     ``weight = weight - learning_rate * gradient``

We can implement this using simple Python code:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different
update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
To enable this, we built a small package: ``torch.optim`` that
implements all these methods. Using it is very simple:



In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

.. Note::

      Observe how gradient buffers had to be manually set to zero using
      ``optimizer.zero_grad()``. This is because gradients are accumulated
      as explained in the `Backprop`_ section.

